<a href="https://colab.research.google.com/github/Valeeew/Data_management_of_an_online_store/blob/master/Data_management_of_an_online_store.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Management & Optimisation d'une boutique en ligne
## Par Valentin MONTEIRO 
---
### Sommaire
> ### IMPORT
- [Import des library](#scrollTo=tAgUgPb2dUt3&line=1&uniqifier=1)
- [Import des DATA](#scrollTo=LGGkJyQgd2Vc&line=3&uniqifier=1)
- Traitement des DATA
>> - [df_web](#scrollTo=THwlX1u91f2t&line=4&uniqifier=1)
>> - [df_liaison](#scrollTo=ykl7KSWzVFsA&line=1&uniqifier=1)❓
- [Lexique des données](#scrollTo=oG89DAzeosCW&line=1&uniqifier=1)

> ### Objectifs
- [Objectifs](#scrollTo=zVQGfyVrRI86&line=15&uniqifier=1)

> ### Code
>> - [Rapprocher les 2 exports](#scrollTo=VpQfY-tkWMYh&line=2&uniqifier=1)
>> - [Calcul CA (turnover) par produit](#scrollTo=nGntr2d-xQYs&line=1&uniqifier=1)
>> - [Calcul du CA (turnover) des ventes en ligne](#scrollTo=dwOAg5jrtGl2&line=1&uniqifier=1)



## IMPORT

---
> ### Import des Libraries
---



In [1]:
import pandas as pd

---
> ### Import des DATA from Github
---




In [15]:
# Clone the entire repo & import DATA
!git clone -l -s git://github.com/Valeeew/Data_management_of_an_online_store cloned-repo
%cd cloned-repo
!ls

# Enregistrement des DATA dans les DATAframe
df_erp = pd.read_excel("erp.xlsx")
df_liaison = pd.read_excel("liaison.xlsx")
df_web = pd.read_excel("web.xlsx")

Cloning into 'cloned-repo'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 18 (delta 9), reused 4 (delta 1), pack-reused 0
Receiving objects: 100% (18/18), 347.94 KiB | 1.82 MiB/s, done.
Resolving deltas: 100% (9/9), done.
/content/cloned-repo/cloned-repo/cloned-repo
Data_management_of_an_online_store.ipynb  erp.xlsx  liaison.xlsx  web.xlsx


---
> ### Traitement des DATA
---




#### df_web

In [16]:
# Traitement des données web inutiles
df_web = df_web.drop(["virtual","downloadable","rating_count","average_rating","tax_class","post_content","comment_status","ping_status","post_password","post_content_filtered","post_parent","menu_order","comment_count"], axis = 1)
# Remplacement des données NaN dans df_web.total_sales
df_web.total_sales = df_web.total_sales.fillna(0)
# Remplacement des données tax_status ➡️ taxable = 1 et Nan = 0
df_web = df_web[df_web.tax_status.notnull()]
#df_web.tax_status = df_web.tax_status.replace("taxable",1)

#### df_liaison

In [22]:
#Suppression des valeurs NaN dans id_web ❓
#df_liaison = df_liaison[df_liaison.id_web.notnull()]

---
> ### Lexique des données
---





#### df_erp

| Variable | Description | Type | PK |
| :--------------- |:---------------| :-----| :---|
| product_id | références produit | int | ✅ |
| onsale_web | vendu en ligne | boolean ||
| price | prix de vente | float ||
| stock_quantity | quantité dans le stock | int ||
|stock_status | État de stock | str ||

#### df_web

| Variable | Description | Type | PK |
| :--------------- |:---------------| :-----| :---|
| sku | références produit | int | ✅ |
| total_sales | nombre de vente | Int ||
| tax_status | produit taxable ou non | Boolean ||
| post_author | auteur de l'article | int ||
| post_date | Date de publication | date ||
| post_date_gmt | Date de publication h GMT| date ||
| post_title | titre de l'article | str ||
| post_excerpt | description de l'article | str ||
| post_status | statut de l'article | str ||
| post_name | Nom de l'article | str ||
| post_modified | Date dernière MAJ | date ||
| post_modified_gmt | Date dernière MAJ h GMT | date ||
| guid | lien | str ||
| post_type | type de l'article | str ||
| post_mime_type | Chemin de l'image | str ||

---
> ### Objectifs
---

- Rapprocher les 2 exports
- Chiffre d’affaires par produit
- Total du chiffre d’affaires réalisé en ligne.
- Analyser la variable price
- - Détecter d’éventuelles valeurs aberrantes
- - Lister les valeur aberrantes 
- - Faire une représentation graphique

---
> ### Code
---

---
>#### Rapprocher les 2 exports
---

In [17]:
# Merge df_erp & df_liaison
df_erp_liaison = df_erp.merge(df_liaison, how="inner", right_on="product_id", left_on="product_id")
# Merge df_erp_liaison & df_web
df_bottleneck = df_erp_liaison.merge(df_web, how="inner", right_on="sku", left_on="id_web")

,product_id,onsale_web,price,stock_quantity,stock_status,id_web
0,3847,1,24.2,0,outofstock,15298
1,3849,1,34.3,0,outofstock,15296
2,3850,1,20.8,0,outofstock,15300
3,4032,1,14.1,0,outofstock,19814
4,4039,1,46.0,0,outofstock,19815
...,...,...,...,...,...,...
820,7203,0,45.0,30,instock,NaN
821,7204,0,45.0,9,instock,NaN
822,7247,1,54.8,23,instock,13127-1
823,7329,0,26.5,14,instock,14680-1


---
>#### Calcul du CA (turnover) produit
---

In [18]:
df_bottleneck["turnover"] = df_bottleneck.price * df_bottleneck.total_sales

---
> #### Calcul du CA (turnover) des vente en ligne
---

In [22]:
df_bottleneck_online = df_bottleneck[(df_bottleneck.onsale_web == 1) == True]
turnover_online = df_bottleneck_online.turnover.sum()

70568.6